# 一等函数

**一等对象** :

+ 在运行时创建
+ 能赋值给变量或者数据结构中的元素
+ 能作为参数传给函数
+ 能作为函数的返回结果

## 把函数视作对象

如下函数显示了其”一等“本性。

In [1]:
def factorial(n):
    """return n!"""
    return 1 if n < 2 else n * factorial(n-1)

factorial(12)

479001600

In [3]:
factorial.__doc__

'return n!'

In [5]:
fn = factorial
fn

<function __main__.factorial>

In [6]:
fn(3)

6

In [7]:
# 函数可以作为一个参数
map(fn, range(10))

In [8]:
list(map(fn, range(10)))

[1, 1, 2, 6, 24, 120, 720, 5040, 40320, 362880]

## 高阶函数

定义：接受函数作为参数、或者返回函数的

In [9]:
fruits = ['strawberry', 'fig', 'apple', 'cherry', 'raspberry', 'banana']
sorted(fruits, key=len)

['fig', 'apple', 'cherry', 'banana', 'raspberry', 'strawberry']

In [10]:
sorted(fruits, key=lambda x: x[::-1])

['banana', 'apple', 'fig', 'raspberry', 'strawberry', 'cherry']

### map,filter, reduce的替代品

map和filter可以使用生成器表达式替代，相同逻辑也可以使用列表生成式进行替代。

reduce在Python3之后放在`functools`里面

## 匿名函数

匿名函数的定义体中不能赋值

## 可调用对象

+ 用户自定义函数
+ 内置函数
+ 内置方法:C语言实现的方法
+ 方法: 类的定义体重定义的函数
+ 类: 调用类相当于调用了__init__函数
+ 类的实例: 如果类定义了__call__方法，那么它的实例可以作为函数调用
+ 生成器函数: 使用yield关键字的函数或方法，调用生成器函数返回的是生成器对象.

## 用户定义的可调用类型

任何python对象都可以表现的像函数，只需要实现实例方法__call__ 即可

In [11]:
import random

class BingoCage:
    
    def __init__(self, items):
        self._items = list(items)
        random.shuffle(self._items)
        
    def pick(self):
        try:
            return self._items.pop()
        except:
            raise LookupError('pick from empty BingoCage')
            
    def __call__(self):
        return self.pick()

In [12]:
bingo = BingoCage(range(3))
bingo

In [13]:
bingo.pick()

1

In [14]:
bingo()

0

In [15]:
callable(bingo)

True

In [16]:
bingo()

2

In [17]:
bingo()

LookupError: pick from empty BingoCage

## 函数内省



In [18]:
dir(factorial)

['__annotations__',
 '__call__',
 '__class__',
 '__closure__',
 '__code__',
 '__defaults__',
 '__delattr__',
 '__dict__',
 '__dir__',
 '__doc__',
 '__eq__',
 '__format__',
 '__ge__',
 '__get__',
 '__getattribute__',
 '__globals__',
 '__gt__',
 '__hash__',
 '__init__',
 '__init_subclass__',
 '__kwdefaults__',
 '__le__',
 '__lt__',
 '__module__',
 '__name__',
 '__ne__',
 '__new__',
 '__qualname__',
 '__reduce__',
 '__reduce_ex__',
 '__repr__',
 '__setattr__',
 '__sizeof__',
 '__str__',
 '__subclasshook__']

In [19]:
class C: pass
obj = C()
def func(): pass

sorted(set(dir(func)) - set(dir(obj)))

['__annotations__',
 '__call__',
 '__closure__',
 '__code__',
 '__defaults__',
 '__get__',
 '__globals__',
 '__kwdefaults__',
 '__name__',
 '__qualname__']

## 从定位参数到仅限关键字参数



In [20]:
def tag(name, *content, cls=None, **attrs):
    """生成一个或多个html标签"""
    
    if cls is not None:
        attrs['class'] = cls
    if attrs:
        attrs_str = ''.join(' %s="%s"' % (attr, value) for attr,value 
                           in sorted(attrs.items()))
    else:
        attrs_str = ''
        
    if content:
        return '\n'.join('<%s%s>%s</%s>' %
                        (name, attrs_str, c, name) for c in content)
    else:
        return '<%s%s />' % (name, attrs_str)

In [21]:
tag('br')

'<br />'

In [22]:
tag('p','hello')

'<p>hello</p>'

In [24]:
print(tag('p','hello','world'))

<p>hello</p>
<p>world</p>


In [26]:
# 没有指明名称的关键字参数会被**attrs捕获
print(tag('p', 'hello', 'world', id=33))

<p id="33">hello</p>
<p id="33">world</p>


In [27]:
print(tag('p', 'hello', 'world', cls='sidebar'))

<p class="sidebar">hello</p>
<p class="sidebar">world</p>


In [28]:
tag(content='testing', name='img')

'<img content="testing" />'

In [29]:
my_tag = {'name':'img', 'title': 'Sunset Boulevard',
         'src':"sunset.jpg", 'cls':'framed'}
tag(**my_tag)

'<img class="framed" src="sunset.jpg" title="Sunset Boulevard" />'

In [30]:
my_tag = {'name':'img', 'title': 'Sunset Boulevard',
         'src':"sunset.jpg", 'cls':'framed','content':['asas','qwqw']}
tag(**my_tag)

'<img class="framed" content="[\'asas\', \'qwqw\']" src="sunset.jpg" title="Sunset Boulevard" />'

## 获取关于参数的信息



In [33]:
import bobo

@bobo.query('/')
def hello(person):
    return 'Hello %s!' % person

## 支持函数式编程的包

### operator模块

在函数式编程中，经常需要将算术运算符当做函数使用。

`operator`模块为多个算数运算符提供了对应的函数，从而避免编写过多的lambda表达式

In [34]:
from functools import reduce
from operator import mul

def fact(n):
    return reduce(mul, range(1, n+1))

`operator`模块还有一类函数，能替代从序列中取出元素或读取对象属性的lambda表达式: `itemgetter`,`attrgetter`

In [35]:
metor_data = [
    ('Tokyo', 'JP', 36.933, (35.67899, 139.212)),
    ('Delhi NCR', 'IN', 21.935, (28.121212, 77.89898)),
]

from operator import itemgetter, attrgetter

for city in sorted(metor_data, key=itemgetter(1)):
    print(city)

('Delhi NCR', 'IN', 21.935, (28.121212, 77.89898))
('Tokyo', 'JP', 36.933, (35.67899, 139.212))


In [36]:
cc_name = itemgetter(1, 0)
for city in metor_data:
    print(cc_name(city))

('JP', 'Tokyo')
('IN', 'Delhi NCR')


In [37]:
from operator import methodcaller

s = 'The time has come'
upcase = methodcaller('upper')
upcase(s)

'THE TIME HAS COME'

In [38]:
hiphenate = methodcaller('replace', " ", "-")
hiphenate(s)

'The-time-has-come'

### 使用functools.partial冻结参数

使用这个函数可以吧接受一个或多个参数的函数改编成需要回调的API,这样参数更少。

In [39]:
from operator import mul
from functools import partial
triple = partial(mul, 3)
triple(7)

21

In [6]:
def func(x,y):
    """测试数据集
    
    Parameter
    --------
    x: x
    y: y
    
    Return
    ------
    x+y
    """
    tmp = 0
    if i<x:
        tmp+=1
    return x+y

In [7]:
func.__annotations__

{}

In [8]:
func.__code__

<code object func at 0x10b719d20, file "<ipython-input-6-ab897342cadd>", line 1>

In [10]:
func.__code__.co_argcount

2

In [11]:
func.__code__.co_varnames

('x', 'y', 'tmp')

In [12]:
func.__code__.co_cellvars

()

In [14]:
x=[1,2,3,4]
x

[1, 2, 3, 4]

In [15]:
from operator import isub

In [17]:
isub?

Docstring: a = isub(a, b) -- Same as a -= b.
Type:      builtin_function_or_method


In [18]:
s='asaas'
s

'asaas'